# XGMIX notebook

In [58]:
import numpy as np
import os

In [59]:
%load_ext autoreload
%autoreload 2

from preprocess import data_process,simulate_missing_values
from XGMIX import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Main arguments

In [60]:
# args = {'train_data': "/home/arvindsk/XGMix/expts/xgmix_global_1/train.txt", 
#         "val_data" : "/home/arvindsk/XGMix/expts/xgmix_global_1/val.txt", 
#         "save": None,
#         "window_size": 1000,
#         "smooth_size": 75,
#         "missing":0.0}

In [61]:
args = {'train_data': "/home/arvindsk/XGMix/expts/xgmix_global_22/train.txt", 
        "val_data" : "/home/arvindsk/XGMix/expts/xgmix_global_22/val.txt", 
        "save": "/home/arvindsk/XGMix/expts/xgmix_global_22/",
        "window_size": 8000,
        "smooth_size": 75,
        "missing":0}

## Pre-processing

In [62]:
class Struct:
    def __init__(self, **entries):
        self.__dict__.update(entries)
args = Struct(**args)

win = args.window_size
training, training_labels, validation, validation_labels = data_process(args)
chmlen = training.shape[-1]
sws = args.smooth_size if args.smooth_size%2 else args.smooth_size-1
num_anc = len(np.unique(training_labels))

Loading data...
Reading /home/arvindsk/datagen/simulation-output/chm22/xgmix_worldwide_22_train/gen_2//mat_vcf_2d.npy
Reading /home/arvindsk/datagen/simulation-output/chm22/xgmix_worldwide_22_train/gen_4//mat_vcf_2d.npy
Reading /home/arvindsk/datagen/simulation-output/chm22/xgmix_worldwide_22_train/gen_6//mat_vcf_2d.npy
Reading /home/arvindsk/datagen/simulation-output/chm22/xgmix_worldwide_22_train/gen_2//mat_map.npy
Reading /home/arvindsk/datagen/simulation-output/chm22/xgmix_worldwide_22_train/gen_4//mat_map.npy
Reading /home/arvindsk/datagen/simulation-output/chm22/xgmix_worldwide_22_train/gen_6//mat_map.npy
Reading /home/arvindsk/datagen/simulation-output/chm22/xgmix_worldwide_22_valid/gen_2//mat_vcf_2d.npy
Reading /home/arvindsk/datagen/simulation-output/chm22/xgmix_worldwide_22_valid/gen_4//mat_vcf_2d.npy
Reading /home/arvindsk/datagen/simulation-output/chm22/xgmix_worldwide_22_valid/gen_6//mat_vcf_2d.npy
Reading /home/arvindsk/datagen/simulation-output/chm22/xgmix_worldwide_22_v

In [63]:
if args.missing != 0:
    training = simulate_missing_values(training,args.missing)
    validation = simulate_missing_values(validation,args.missing)

In [64]:
try:
    os.mkdir(args.save)
except:
    print("Directory exists")

Directory exists


In [65]:
xgm = XGMIX(chmlen,win,sws,num_anc,args.save,cores=16)

In [66]:
xgm.train(training,training_labels,validation, validation_labels, smoothlite=10000)

Windows done: 0, 
Base Training Accuracy: 0.9844444444444445, Base Validation Accuracy: 0.8233333333333334
Smooth Training Accuracy: 1.0 
Smooth Validation Accuracy: 0.9287179487179488 


In [ ]:
# predict using the built-in method.
ypreds = xgm.predict(validation)
print("Accuracy")
print(np.mean(ypreds == validation_labels.numpy()))
print("Confusion matrix")
print(sklearn.metrics.confusion_matrix(ypreds.ravel(), validation_labels.numpy().ravel()))

In [ ]:
# predict using the standalone function.
ypreds = predict(validation,args.save+"/model.pkl")
print("Accuracy")
print(np.mean(ypreds == validation_labels.numpy()))
print("Confusion matrix")
print(sklearn.metrics.confusion_matrix(ypreds.ravel(), validation_labels.numpy().ravel()))